<a href="https://colab.research.google.com/github/emmanuellfc/EM_Tufts_Fall2023/blob/main/GeneralFiniteDifferences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Mount Google Drive.
# For getting files from drive.
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks')

In [ ]:
# @title Dependency nonsense.

%%capture

# Package dependencies.
!apt install libcgal-dev libeigen3-dev
!pip install -U pygalmesh
!pip install networkx

#import pygalmesh
#import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from typing import Iterable
from typing import Callable
from typing import Optional
import math
import csv
import sys
try: del sys.modules['GeneralFiniteDifferences'] # Terrible hack to force reload GFDM because I'm still editing it.
except KeyError: pass
from GeneralFiniteDifferences import FiniteDifferences
from GeneralFiniteDifferences import MeshPoint

# FEM importing.
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
from fenics import *
from mshr import *
# Plotting Libraries.
plt.style.use("_classic_test_patch")

The following are:
  - The **FiniteDifferences** class which encapsulates the General Finite Differences Method (GFDM) solver
  - Objects that are shared between sub-experiments in this notebook, including plotting macros and shared boundary conditions

In [ ]:
# @title Useful things for analysis to consolidate common code (plotting shortcuts, shape generators, and boundary conditions).

# Plot a GFDM output.
def draw_gfdm_solution(solution:FiniteDifferences):
  fig, ax = plt.subplots()
  ax.set_aspect('equal')
  plt.scatter([ pt.pos[0] for pt in solution.all_points ],
              [ pt.pos[1] for pt in solution.all_points ],
            c=[ pt.pot    for pt in solution.all_points ],
       marker='.')
  cbar = plt.colorbar()
  cbar.set_label('GFDM potential')
  plt.show()

# Plot an electric field vector field for a GFDM output.
def draw_gfdm_electric(solution:FiniteDifferences):
  fig,ax = plt.subplots(dpi=150)
  ax.set_aspect('equal')
  plt.quiver([ pt.pos[0]               for pt in solution.all_points ],
             [ pt.pos[1]               for pt in solution.all_points ],
             [ pt.elec[0]              for pt in solution.all_points ],
             [ pt.elec[1]              for pt in solution.all_points ],
             [ np.linalg.norm(pt.elec) for pt in solution.all_points ])
  cbar = plt.colorbar()
  cbar.set_label('||Electric field||')
  plt.show()

# Generic error function for some analytic function.
def generic_error(point:MeshPoint,analytic_func:Callable[[MeshPoint],Optional[float]]) -> float:
  analytic = analytic_func(point)
  if analytic is None: return 0
  else: return point.pot - analytic_func(point)

# Return a callable that gets the solution for a FEM solution (this helps clean up the syntax when we treat FEM as the analytic solution).
def fem_pot(fem_solution) -> Callable[[MeshPoint],Optional[float]]:
  # If you are reading this please don't look at what I'm about to do.
  def solution_eval(mesh_point:MeshPoint) -> float:
    pt = Point(mesh_point.pos[0],mesh_point.pos[1])
    try:
      return fem_solution(pt)
    except RuntimeError:
      return None
  return solution_eval
  #return lambda mesh_point: fem_solution(Point(mesh_point.pos[0],mesh_point.pos[1]))

# Plot a generic error plot between GFDM and some accepted solution.
def draw_gfdm_error(solution:FiniteDifferences,analytic_func:Callable[[MeshPoint],Optional[float]]):
  fig, ax = plt.subplots()
  ax.set_aspect('equal')
  plt.scatter([ pt.pos[0]                       for pt in solution.all_points ],
              [ pt.pos[1]                       for pt in solution.all_points ],
            c=[ generic_error(pt,analytic_func) for pt in solution.all_points ],
       marker='.')
  cbar = plt.colorbar()
  cbar.set_label('Absolute error')
  plt.show()

# Plot an FEM solution.
def draw_fem_solution(fem_solution):

  fig,ax = plt.subplots()
  ax.set_aspect('equal')
  plot(fem_solution)
  cbar = plt.colorbar(plot(fem_solution))
  cbar.set_label('FEM potential')
  plt.show()

  """
  plt.figure()                       # Set the resolution
  plot(fem_solution, title = 'Potential Field')              # Set the title
  plt.colorbar(plot(fem_solution)) # Add a colorbar
  plt.show()
  """

# Load a mesh into a list of (x,y) tuples from a CSV file in Google Drive.
def load_mesh_from_csv(name:str) -> list:
  ret = []
  with open(name) as f:
    for row in csv.reader(f): ret.append( (float(row[0]),float(row[1])) )
  return ret

# Coax analytic solution.
def coax_analytic_pot_r(r:float) -> float:
  V_0 = 10.
  a = 0.5
  b = 2.
  if r <= a: return V_0 # Inside the inner conductor, we just have the potential of that inner conductor.
  if r >= b: return 0 # Outside the outer conductor, we just have zero (Gauss's Law - no net charge).
  return -V_0/np.log(b/a)*np.log(r/a) + V_0 # numpy is silly and has log as base e.

def coax_analytic_pot(point:MeshPoint) -> float:
  return coax_analytic_pot_r( np.linalg.norm(point.pos) )

# Coax geometries.
def circle_generator(radius=1,npoints=24):
  for i in range(0,npoints):
    x = radius * math.cos(2*math.pi * i/npoints)
    y = radius * math.sin(2*math.pi * i/npoints)
    yield (x,y)

def circle_boundary(point: MeshPoint) -> Optional[float]:
  radius = np.linalg.norm(point.pos)
  if radius <= 0.5: return 10
  if radius >= 2: return 0
  return None

# Conducting bars geometries.
def bars_generator():
  yield (-0.10,-0.10)
  yield (-0.10,6.10)
  yield (6.10,6.10)
  yield (6.10,-0.10)

def bars_boundary(point: MeshPoint) -> Optional[float]:
  # Each of the bars are held at 2.0 V.
  if point.pos[0] >= 1.0 and point.pos[0] <= 5.0:
    if ( point.pos[1] >= 1.0 and point.pos[1] <= 1.5 ) or ( point.pos[1] >= 4.5 and point.pos[1] <= 5.0 ): return 10.0
  # The bounding box is grounded.
  if point.pos[0] <= 0.0 or point.pos[0] >= 6.0: return 0.0
  if point.pos[1] <= 0.0 or point.pos[1] >= 6.0: return 0.0
  return None

# Given a mesh list and exact boundary list, return a new boundary that smooths out the old boundary (includes nearby points from the mesh).
def make_fuzzy_boundary(mesh_coords:list,boundary_coords:list,fudge:float) -> list:
  ret = []
  for mesh_pt in mesh_coords:
    for boundary_pt in boundary_coords:
      if np.linalg.norm([mesh_pt[0] - boundary_pt[0],mesh_pt[1] - boundary_pt[1]]) <= fudge:
        ret.append(mesh_pt)
        break
  return ret


Baby steps: Make a solution to a cylindrical (2D projection) of a coaxial cable using the GFDM method.

In [ ]:
# @title Make our best solution to the COAX problem.

# Make a new solver object.
solution = FiniteDifferences()
# Tell it how large to make the max edge size and how many neighbors we want.
solution.set_max_edge_size(0.20)
solution.set_num_neighbors(6)

# Tell it about our boundary condition function.
solution.inform_with_boundary_conditions(circle_boundary)
# Have the solver sample points within the edges.
solution.make_points_from_boundary(circle_generator(radius=2.5,npoints=36))
# Form the internal graph data structure.
solution.make_graph_from_points()
# Generate the systems matrix that represents the linear set of equations to solve.
solution.make_systems_matrix()
# Tell the solver to try and solve the systems matrix.
solution.solve_systems_matrix()
# Also compute the electric field for this solution.
solution.compute_electric_field()
# Debug: Draw the connectivity graph.
solution.draw_connectivity()

In [ ]:
# @title Plot the GFDM solution, the electric field for GFDM, and the analytic solution.
# Plot an analytic radial solution for one cylinder inside another (coax cable). We reduce the problem to 2D by ignoring the z axis.

cartesian_min = -2.5
cartesian_max = 2.5
nsteps = 300
x_values = np.linspace(cartesian_min,cartesian_max,nsteps)
y_values = np.linspace(cartesian_min,cartesian_max,nsteps)
z_values = np.array([ coax_analytic_pot_r(np.linalg.norm([x,y])) for x in x_values for y in y_values ])

x_coords, y_coords = np.meshgrid(x_values,y_values)
z_coords = z_values.reshape(nsteps,nsteps)

x_limits = [cartesian_min,cartesian_max]
y_limits = [cartesian_min,cartesian_max]

fig, ax = plt.subplots()
plt.pcolor(x_coords,y_coords,z_coords)
ax.set_xlim(x_limits)
ax.set_ylim(y_limits)
cbar = plt.colorbar()
cbar.set_label('Analytic potential')
plt.show()

# Plot the solved potential (again) with color indicating potential at that point.
draw_gfdm_solution(solution)

# Plot the electric field.
draw_gfdm_electric(solution)

# Plot the error between GFDM and the analytic COAX solution.
draw_gfdm_error(solution,coax_analytic_pot)

print(f'Root mean square error: {solution.get_rms_error(coax_analytic_pot)}')

Establish RMS as an error index and use it to quantify how choices of node density and neighbor count impact the goodness of the GFDM solution. *This step is slow and can take more than 30 minutes.*

In [ ]:
# @title Compute RMS error for various COAX GFDM choices of hyper-parameters.
# Using the same geometry as before, find the root mean square error as a function of node count... (This one will take awhile.)

max_edge_sizes = [ 0.80, 0.75, 0.70, 0.65, 0.60, 0.55, 0.50, 0.45, 0.40, 0.35, 0.30, 0.25, 0.20, 0.16, 0.12, 0.10, 0.09, 0.08] # Edge sizes we should solve for.
num_neighbors = [ 4, 6, 8, 10, 16 ] # Neighbor counts we should solve for.
# Make a collection of solution objects and set each up independently...
coax_solutions = [] # 2D array: First index is neighbor count, second index is edge size.

print(f'Making {len(max_edge_sizes)*len(num_neighbors)} new solution solver objects.')
for neighbors in num_neighbors:
  new_row = [] # Make a new row for this neighbor count.
  for max_edge_size in max_edge_sizes:
    new_solution = FiniteDifferences()
    new_solution.set_hush(True)
    new_solution.set_max_edge_size(max_edge_size)
    new_solution.set_num_neighbors(neighbors)
    new_row.append(new_solution)
  coax_solutions.append(new_row) # Now that it's been assembled, add the new row to the 2D array.

print('Building meshes for all solutions and informing with boundary conditions.')
for row in coax_solutions:
  for new_solution in row:
    new_solution.inform_with_boundary_conditions(circle_boundary)
    new_solution.make_points_from_boundary(circle_generator(radius=2.5,npoints=36))

print('Building connectivity graphs for all solutions (this is the slow step).')
for row in coax_solutions:
  for new_solution in row:
    new_solution.make_graph_from_points()

print('Building systems matrices for all solutions.')
for row in coax_solutions:
  for new_solution in row:
    new_solution.make_systems_matrix()

print('Solving systems matrices for all solutions.')
for row in coax_solutions:
  for new_solution in row:
    new_solution.solve_systems_matrix()

# We now have solutions for all of our systems!
print(f'Done solving {len(max_edge_sizes)*len(num_neighbors)} systems!')

In [ ]:
# @title Plot RMS error for the previous cell.
# Make plotting a separate cell so that we don't need to re-evaluate solutions to change the plotting style.

# Plot RMS error versus the number of nodes in the plot.
scatters = []
fig, ax = plt.subplots()
for i,neighbors in enumerate(coax_solutions):
  scatter = ax.scatter([ len(solution.all_points) for solution in coax_solutions[i] ],
                       [ solution.get_rms_error(coax_analytic_pot) for solution in coax_solutions[i] ],
                       label = f'{neighbors} neighbors')
  scatters.append(scatter)
ax.legend(handles=scatters,labels=[ f'{neighbors} neighbors' for neighbors in num_neighbors ],loc='upper right')
ax.set_xlabel('Number of nodes in mesh')
ax.set_ylabel('RMS error')
plt.show()

How well does the GFDM solution hold up as a function of radius? Average the GFDM solution over concentric circles and compare these targets to the analytic solution at these radii.

In [ ]:
# @title Compute a solution to the COAX problem for GFDM radial averaging.
# Make a new solution that we'll use for our radial pot average plot.

solution_r = FiniteDifferences()
solution_r.set_max_edge_size(0.07)
solution_r.set_num_neighbors(6)
solution_r.inform_with_boundary_conditions(circle_boundary)
solution_r.make_points_from_boundary(circle_generator(radius=2.5,npoints=36))
solution_r.make_graph_from_points()
solution_r.make_systems_matrix()
solution_r.solve_systems_matrix()

In [ ]:
# @title Plot radial GFDM for the previous cell.
# Build a plot that shows the potential as a function of radius. For some radius, take an average of potential within a concentric ring...
radstart = 0.5
radend = 2.0
nradii = 50
radii = np.linspace(radstart,radend,nradii)

# Helper function that smooths out the pot over a target radius with some width.
def get_radial_average(solution:FiniteDifferences,target:float,width=0.05) -> float:
  pot_to_average = []
  for pt in solution.all_points:
    radius = np.linalg.norm(pt.pos)
    if target-width/2 <= radius <= target+width/2:
      pot_to_average.append(pt.pot)
  return np.mean(pot_to_average)

smooth_radii = np.linspace(radstart,radend,1000)
numerical_by_radius = [ get_radial_average(solution_r,radius) for radius in radii ]

fig, ax = plt.subplots()
numerical, = plt.plot(radii,numerical_by_radius)
analytic, = plt.plot(smooth_radii,[ coax_analytic_pot_r(radius) for radius in smooth_radii ])
ax.legend(handles=[numerical,analytic],labels=[ 'General finite differences','Analytic' ],loc='upper right')
ax.set_xlabel('Radius')
ax.set_ylabel('Potential')
plt.show()

So far, we have allowed the GFDM solver to make its own mesh. How well does it do when we force it to use a mesh produced by FEniCS?

In [ ]:
# @title Compute the GFDM COAX solution using the mesh produced by fenics.

# Make a new COAX solution but force the solver to use the mesh generated by fenics for a one-to-one comparison. (File is in drive as a CSV...)
fenics_coax_mesh = load_mesh_from_csv('Coaxial_Coordinates.csv')

solution_fenics_coax = FiniteDifferences()
solution_fenics_coax.set_num_neighbors(6)
solution_fenics_coax.set_max_edge_size(0.20)
solution_fenics_coax.inform_with_boundary_conditions(circle_boundary)
solution_fenics_coax.inform_with_manual_mesh(fenics_coax_mesh)
solution_fenics_coax.make_graph_from_points()
solution_fenics_coax.make_systems_matrix()
solution_fenics_coax.solve_systems_matrix()

In [ ]:
# @title Plot the GFDM COAX solution using the fenics mesh from the previous cell.

# Plot the solution itself.
draw_gfdm_solution(solution_fenics_coax)

# Plot absolute error between GFDM and the analytic solution.
draw_gfdm_error(solution_fenics_coax,coax_analytic_pot)

# Also draw connectivity
solution_fenics_coax.draw_connectivity()

print(f'Root mean square error: {solution_fenics_coax.get_rms_error(coax_analytic_pot)}')


New problem: Solve two finite-length conducting bars using the GFDM.

In [ ]:
# @title Compute the GFDM solution for the two bars.

solution_bars = FiniteDifferences()
solution_bars.set_num_neighbors(6)
solution_bars.set_max_edge_size(0.13)
solution_bars.inform_with_boundary_conditions(bars_boundary)
solution_bars.make_points_from_boundary(bars_generator())
solution_bars.make_graph_from_points()
solution_bars.make_systems_matrix()
solution_bars.solve_systems_matrix()

In [ ]:
# @title Compute the FEM solution for the two bars.

%%capture

x_lenght = 6.0
y_lenght = 6.0
ambient = Rectangle(Point(0.0, 0.0), Point(x_lenght, y_lenght))
barone  = Rectangle(Point(1.0, 1.0), Point(5.0, 1.5))
bartwo = Rectangle(Point(1.0, 4.5), Point(5.0, 5.0))
domain = ambient - barone - bartwo
mesh = generate_mesh(domain, 64)

# Sub domain for no-slip (mark whole boundary, inflow and outflow will overwrite)
class Interior(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

# Sub domain for inflow (right)
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return bool( near(x[0], 0.0) and on_boundary)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return bool( near(x[0], x_lenght) and on_boundary)

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return bool( near(x[1], 0.0) and on_boundary)

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return bool( near(x[1], y_lenght) and on_boundary)

# Create mesh functions over the cell facets
sub_domains = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
# Mark all facets as sub domain 5
sub_domains.set_all(5)

interior = Interior()
left     = Left()
right    = Right()
bottom   = Bottom()
top      = Top()

interior.mark(sub_domains, 0)
left.mark(sub_domains, 1)
right.mark(sub_domains, 2)
bottom.mark(sub_domains, 3)
top.mark(sub_domains, 4)

V = FunctionSpace(mesh, 'P', 1)
# Boundary Conditions
bc0 = DirichletBC(V, Constant(10.0), sub_domains, 0)
bc1 = DirichletBC(V, Constant(0.0), sub_domains, 1)
bc2 = DirichletBC(V, Constant(0.0), sub_domains, 2)
bc3 = DirichletBC(V, Constant(0.0), sub_domains, 3)
bc4 = DirichletBC(V, Constant(0.0), sub_domains, 4)
bcs = [bc0, bc1, bc2, bc3, bc4]

u_bars = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(u_bars), grad(v)) * dx
L = Constant('0') * v * dx
u_bars = Function(V)
solve(a == L, u_bars, bcs)

In [ ]:
# @title Plot the two bars solution from the previous cell.

draw_gfdm_solution(solution_bars)

solution_bars.draw_connectivity()

In [ ]:
# @title Plot the GFDM vs FEM error.

# First plot the FEM solution.
draw_fem_solution(u_bars)

# Draw the absolute error plot taking FEM as accepted.
draw_gfdm_error(solution_bars,fem_pot(u_bars))

print(f'Root mean square error: {solution_bars.get_rms_error(fem_pot(u_bars))}')


Next, force the GFDM solver to use a mesh produced by FEniCS. How well does it do compared to the FEM?

In [ ]:
# @title Compute the GFDM two bars solution using the mesh produced by fenics.

# Make a new COAX solution but force the solver to use the mesh generated by fenics for a one-to-one comparison. (File is in drive as a CSV...)
fenics_bars_mesh = load_mesh_from_csv('Two_Bars_Vertices.csv')

solution_fenics_bars = FiniteDifferences()
solution_fenics_bars.set_num_neighbors(6)
solution_fenics_bars.set_max_edge_size(0.20)
solution_fenics_bars.inform_with_boundary_conditions(bars_boundary)
solution_fenics_bars.inform_with_manual_mesh(fenics_bars_mesh)
solution_fenics_bars.make_graph_from_points()
solution_fenics_bars.make_systems_matrix()
solution_fenics_bars.solve_systems_matrix()

In [ ]:
# @title Plot the GFDM two bars solution from the previous cell.
draw_gfdm_solution(solution_fenics_bars)

solution_fenics_bars.draw_connectivity()

In [ ]:
# @title Plot the comparison between FEM and GFDM for the two bars.

# Next plot the absolute difference between FEM and GFDM.
draw_gfdm_error(solution_fenics_bars,fem_pot(u_bars))

print(f'Root mean square error: {solution_fenics_bars.get_rms_error(fem_pot(u_bars))}')


Challenge problem: How well does the GFDM do for a highly irregular geometry! How about a dolfin? :)

In [ ]:
# @title Compute the dolfin solution with GFDM using the mesh made by fenics.
import xml.etree.ElementTree as et
tree = et.parse('dolfin_fine.xml')
root = tree.getroot()

# Unpack the XML elemet tree into a list of tuples: [ (x1,y1), (x2,y2), ... ]
vertices = root.find('mesh').find('vertices')
dolfin_mesh_coords = [ ( float(vertex.attrib['x']), float(vertex.attrib['y']) ) for vertex in vertices ]

x_limits = [-0.10,1.10]
y_limits = [-0.10,1.10]

"""
fig, ax = plt.subplots()
plt.scatter([ pt[0] for pt in dolfin_mesh_coords ],
            [ pt[1] for pt in dolfin_mesh_coords ],
            marker='.')
ax.set_xlim(x_limits)
ax.set_ylim(y_limits)
plt.show()
"""

# Very Formal Experiment has shown that the first 77 points are used to make the dolfin outline - use these as BC!
# Are there any other points that are near these points? Fudge the boundary a litle bit to include more points...
dolfin_boundary_coords = make_fuzzy_boundary(dolfin_mesh_coords,dolfin_mesh_coords[:77],0.01)

"""
dolfin_boundary_coords = []
for mesh_pt in dolfin_mesh_coords:
  for boundary_pt in bare_dolfin_boundary_coords:
    if np.linalg.norm([mesh_pt[0] - boundary_pt[0],mesh_pt[1] - boundary_pt[1]]) < 0.01:
      dolfin_boundary_coords.append(mesh_pt)
      break
"""
"""
fig, ax = plt.subplots()
plt.scatter([ pt[0] for pt in dolfin_boundary_coords ],
            [ pt[1] for pt in dolfin_boundary_coords ],
            marker='.')
ax.set_xlim(x_limits)
ax.set_ylim(y_limits)
plt.show()
"""

def dolfin_boundary_func(pt:MeshPoint) -> float:
  if pt.pos[1] <= 0.02: return 5.0 # Bottom boundary is 5.0 V.
  if pt.pos[0] <= 0.02 or pt.pos[0] >= 0.98: return 0.0 # Bounding box grounded.
  if pt.pos[1] >= 0.98: return 0.0 # Bounding box grounded.
  if pt.pos in dolfin_boundary_coords: return 10.0 # Boundary of the dolphin is set to 10.0 V.

print(f'Making a manual dolfin mesh using {len(dolfin_mesh_coords)} mesh and {len(dolfin_boundary_coords)} boundary points...')

dolfin_solution = FiniteDifferences()
dolfin_solution.set_num_neighbors(6)
dolfin_solution.set_max_edge_size(0.10)
dolfin_solution.inform_with_boundary_conditions(dolfin_boundary_func)
dolfin_solution.inform_with_manual_mesh(dolfin_mesh_coords)
dolfin_solution.make_graph_from_points()
dolfin_solution.make_systems_matrix()
dolfin_solution.solve_systems_matrix()


In [ ]:
# @title Solve the dolfin problem using FEM.

%%capture

# Sub domain for no-slip (mark whole boundary, inflow and outflow will overwrite)
class Interior(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

# Sub domain for inflow (right)
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] > 1.0 - DOLFIN_EPS and on_boundary

# Sub domain for outflow (left)
class Right(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < DOLFIN_EPS and on_boundary

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return x[1] < DOLFIN_EPS and on_boundary

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return x[1] > 1.0 - DOLFIN_EPS and on_boundary

mesh = Mesh('dolfin_fine.xml.gz')

# Create mesh functions over the cell facets
sub_domains = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
# Mark all facets as sub domain 5
sub_domains.set_all(5)

interior = Interior()
left     = Left()
right    = Right()
bottom   = Bottom()
top      = Top()

interior.mark(sub_domains, 0)
left.mark(sub_domains, 1)
right.mark(sub_domains, 2)
bottom.mark(sub_domains, 3)
top.mark(sub_domains, 4)

V = FunctionSpace(mesh, 'P', 1)
# Boundary Conditions
bc0 = DirichletBC(V, Constant(10.0), sub_domains, 0)
bc1 = DirichletBC(V, Constant(0.0), sub_domains, 1)
bc2 = DirichletBC(V, Constant(0.0), sub_domains, 2)
bc3 = DirichletBC(V, Constant(5.0), sub_domains, 3)
bc4 = DirichletBC(V, Constant(0.0), sub_domains, 4)
bcs = [bc0, bc1, bc2, bc3, bc4]

u_dolfin = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(u_dolfin), grad(v)) * dx
L = Constant('0') * v * dx
u_dolfin = Function(V)
solve(a == L, u_dolfin, bcs)

In [ ]:
# @title Plot the dolfin solution for GFDM.
dolfin_solution.draw_connectivity()

draw_gfdm_solution(dolfin_solution)

In [ ]:
# @title Plot the FEM solution to the dolfin problem.

# Plotting the solution.
draw_fem_solution(u_dolfin)

In [ ]:
# @title Perform an absolute error comparison between GFDM and FEM for the dolfin.

draw_gfdm_error(dolfin_solution,fem_pot(u_dolfin))

print(f'Root mean square error: {dolfin_solution.get_rms_error(fem_pot(u_dolfin))}')

Another challenge problem for highly irregular geometry...

In [ ]:
# @title Compute the python solution with GFDM using the mesh made by fenics.

# Get the mesh from Google Drive.
python_mesh_coords = load_mesh_from_csv('Pythonic.csv')

"""
fig,ax = plt.subplots()
ax.set_aspect('equal')
plt.scatter( [ pt[0] for pt in python_mesh_coords ],
             [ pt[1] for pt in python_mesh_coords ],
      marker='.')
plt.show()
"""

python_boundary_coords = make_fuzzy_boundary(python_mesh_coords,python_mesh_coords[:433],5)

def python_boundary_func(pt:MeshPoint) -> float:
  # Bounding box is grounded...
  if pt.pos[0] <= 0.02 or pt.pos[0] >= 269.98: return 0.0
  if pt.pos[1] >= -0.02 or pt.pos[1] <= -498.98: return 0.0
  if pt.pos in python_boundary_coords: return 10 # Boundary of the python is set to 10.0 V.

print(f'Making a manual python mesh using {len(python_mesh_coords)} mesh and {len(python_boundary_coords)} boundary points...')

python_solution = FiniteDifferences()
python_solution.set_num_neighbors(6)
python_solution.set_max_edge_size(45)
python_solution.inform_with_boundary_conditions(python_boundary_func)
python_solution.inform_with_manual_mesh(python_mesh_coords)
python_solution.make_graph_from_points()
python_solution.make_systems_matrix()
python_solution.solve_systems_matrix()

In [ ]:
# @title Get an FEM solution for the python.

%%capture

# Sub domain for no-slip (mark whole boundary, inflow and outflow will overwrite)
class Interior(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

# Sub domain for inflow (right)
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] > 250.0 - DOLFIN_EPS and on_boundary

# Sub domain for outflow (left)
class Right(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < DOLFIN_EPS and on_boundary

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return x[1] <  -499.0 - DOLFIN_EPS and on_boundary

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return x[1] > 0.0 - DOLFIN_EPS and on_boundary

mesh = Mesh('snake_hole.xml')

# Create mesh functions over the cell facets
sub_domains = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
# Mark all facets as sub domain 5
sub_domains.set_all(5)

interior = Interior()
left     = Left()
right    = Right()
bottom   = Bottom()
top      = Top()

interior.mark(sub_domains, 0)
left.mark(sub_domains, 1)
right.mark(sub_domains, 2)
bottom.mark(sub_domains, 3)
top.mark(sub_domains, 4)

V = FunctionSpace(mesh, 'P', 1)
# Boundary Conditions
bc0 = DirichletBC(V, Constant(10.0), sub_domains, 0)
bc1 = DirichletBC(V, Constant(0.0), sub_domains, 1)
bc2 = DirichletBC(V, Constant(0.0), sub_domains, 2)
bc3 = DirichletBC(V, Constant(0.0), sub_domains, 3)
bc4 = DirichletBC(V, Constant(0.0), sub_domains, 4)
bcs = [bc0, bc1, bc2, bc3, bc4]

u_python = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(u_python), grad(v)) * dx
L = Constant('0') * v * dx
u_python = Function(V)
solve(a == L, u_python, bcs)

In [ ]:
# @title Plot the python solution for GFDM.
python_solution.draw_connectivity()

draw_gfdm_solution(python_solution)

In [ ]:
# @title Compare GFDM to FEM for the python.

draw_fem_solution(u_python)

draw_gfdm_error(python_solution,fem_pot(u_python))